# Cuaderno 31: Problema de corte de material 
# (Cutting Stock Problem) 

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\S}{\mathcal{S}}$


Dados: 
* un conjunto de rollos, cada uno de longitud $L$; y,
* $m$ órdenes de corte, cada una de las cuales consiste en cortar de los rollos disponibles $b_i$ items de longitud $w_i$, con $w_i < L$, $i \in \{1, \ldots, m\}$.

El problema del corte de material (*Cutting Stock Problem*) consiste en determinar la manera de satisfacer todas las órdenes de corte empleando la menor cantidad posible de rollos. 

Para formular este problema como un programa lineal entero, determinaremos primero una cota superior $N$ al número de rollos necesarios para cumplir con todas las órdenes. Notar que si cortamos cada item de un rollo distinto, necesitamos como máximo $\sum_{i=1}^m b_i$ rollos. Otra cota más ajustada consiste en determinar la cantidad de rollos necesaria para satisfacer cada pedido de manera independiente, en cuyo caso se tiene: 
$$
N:= \sum_{i=1}^m \left\lceil \frac{b_i}{\lfloor \frac{L}{w_i} \rfloor} \right\rceil 
\leq \sum_{i=1}^m \left\lceil \frac{b_i}{\frac{L - w_i}{w_i}} \right\rceil = \sum_{i=1}^m \left\lceil \frac{b_i w_i}{L - w_i} \right\rceil
$$

Definimos ahora dos conjuntos de variables de decisión: las variables binarias $y_j$, con $j \in \{1, \ldots, N\}$,
indican si se usa o no el rollo $j$ en la solución. Las variables enteras $x_{ij}$, por su parte, indican la cantidad de items de la orden $i$ que serán cortados del rollo $j$. Con estas variables, el problema de corte de material puede formularse como el siguiente programa lineal entero:
\begin{align*}
\min &\sum_{j \in J} y_j\\ 
& \mbox{s.r.}\\
&\sum_{j \in J} x_{ij} \geq b_i, \quad \forall i \in I,\\
&\sum_{i \in I} w_i x_{ij} \leq L y_j, \quad \forall j \in J,\\
& x_{ij} \in \ZZ, \quad \forall i \in I, j \in J,\\
& y_j \in \{0, 1\}, \forall j \in J,
\end{align*}
donde $I:= \{1, \ldots, m \}$ es el conjunto de órdenes de corte y $J:= \{1, \ldots, N \}$ es el conjunto de rollos disponibles.


Implementaremos a continuación este modelo utilizando la interfaz Python del solver Gurobi.


Definimos primero los datos. Generaremos $m$ órdenes de corte, consistentes en cortar items de longitud entera aleatoria entre 1 y 9. La cantidad de items a cortar de cada tipo estará dada por un número aleatorio entre 10 y 50. La longitud de los rollos será $L := 10$.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random as rm
import math

# iniciar generador de numeros aleatorios
rm.seed(0)

# número de órdenes
m = 100

# lista con los índices de las órdenes,
# diccionarios con las longitudes y demandas de cada item
I, w, b = gp.multidict({
    i : (rm.randint(1, 9), rm.randint(10, 50)) 
        for i in range(1, m+1)}
)
    
L = 10

# cantidad maxima de rollos a utilizar
N = sum(int(math.ceil(w[i]*b[i]/(L - w[i]))) for i in I)

# conjuntos de índices de rollos
J = range(1, N+1)

print(w)
print(b)
print(N)

Definimos ahora el modelo , sus variables y la función objetivo. 


In [ ]:
# crear el objeto modelo
m = gp.Model('cutting-stock')

# crear las variables asociadas a cantidad de items cortados de cada rollo
x = m.addVars(I, J, name="x", vtype=GRB.INTEGER)

# crear las variables asociadas al uso de rollos
y = m.addVars(J, name="y", vtype=GRB.BINARY)

# crear la función objetivo
m.setObjective(y.sum('*'), GRB.MINIMIZE)


Añadimos las restricciones de satisfacción de la demanda. 

In [ ]:
# restricciones de demanda
m.addConstrs((x.sum(i,'*') >= b[i] for i in I), "demanda");


Añadimos las restricciones de enforzamiento que impiden asignar items a rollos sin utilizar, y garantizan además que la longitud de los rollos sea respetada al cortar los items:

In [ ]:
# restricciones de tamaño de rollos
m.addConstrs((sum([w[i]*x[i,j] for i in I]) <= L*y[j] for j in J), "tam_rollo");


Para encontrar una solución inicial, implementamos una heurística sencilla que consiste en atender los pedidos secuencialmente, es decir, sin mezclar pedidos en un mismo rollo:

In [ ]:
# diccionario con valores de x y y (inicializar en cero)
vx = {(i,j) : 0 for i in I for j in J} 
vy = {j : 0 for j in J}

ind = 0
# satisfacer pedidos secuencialmente
for i in I:
    # print(ind)
    # cantidad de items producidos del pedido i
    producidos = 0
    # cantidad de items que pueden cortarse de un rollo
    cant = int(L // w[i]) 
    # asignar rollos al pedido i hasta saitsfacer la demanda
    while producidos < b[i]:
        j = J[ind]
        vy[j]= 1
        vx[i,j] = cant
        producidos += cant
        ind += 1


Ahora asignamos los valores de la solución heurística (almacenados en los diccionarios `vx` y `vy`) como valores iniciales para las variables del modelo, fijando el atributo `Start`:

In [ ]:
# cargar valores iniciales de x
for i in I:
    for j in J:
        x[i,j].Start = vx[i,j]
        
# cargar valores iniciales de y
for j in J:
    y[j].Start = vy[j]
        

Finalmente, resolvemos el modelo. Fijamos un límite para el tiempo de cálculo y una tolerancia para la brecha de optimalidad:

In [ ]:
# terminar al alcanzar un gap del 10%
m.Params.MIPGap = 0.01

# terminar luego de 180 segundos
m.Params.TimeLimit = 180

# resolver el modelo
m.optimize()

Escribir la solución óptima:

In [ ]:
# recuperar variables y_j cuyo valor sea 1:
for j in J:
    if y[j].getAttr('X') > 0.1:
        # recuperar valores x_ij asociados al rollo j y ponerlos en una lista
        L =[(int(x[i,j].getAttr('X')),i) for i in I if x[i,j].getAttr('X')>=0.9]
        # transformar la lista en una cadena de caracteres e imprimirla
        S = ['{:2}x{} '.format(k,i) for k,i in L]
        print ('[{}]'.format(''.join(S)))


## Código completo

Se reproduce a continuación el código completo del modelo anterior. 

In [ ]:
# Implementación de modelos lineales enteros
# Problema del corte de material (Cutting-Stock)
# Luis M. Torres (EPN 2022)

import gurobipy as gp
from gurobipy import GRB
import random as rm
import math

# iniciar generador de numeros aleatorios
rm.seed(0)

# número de órdenes
m = 100

# lista con los índices de las órdenes,
# diccionarios con las longitudes y demandas de cada item
I, w, b = gp.multidict({
    i : (rm.randint(1, 9), rm.randint(10, 50)) 
        for i in range(1, m+1)}
)
    
L = 10

# cantidad maxima de rollos a utilizar
N = sum(int(math.ceil(w[i]*b[i]/(L - w[i]))) for i in I)

# conjuntos de índices de rollos
J = range(1, N+1)

try:
    # crear el objeto modelo
    m = gp.Model('cutting-stock')

    # crear las variables asociadas a cantidad de items cortados de cada rollo
    x = m.addVars(I, J, name="x", vtype=GRB.INTEGER)

    # crear las variables asociadas al uso de rollos
    y = m.addVars(J, name="y", vtype=GRB.BINARY)

    # crear la función objetivo
    m.setObjective(y.sum('*'), GRB.MINIMIZE)

    # restricciones de demanda
    m.addConstrs((x.sum(i,'*') >= b[i] for i in I), "demanda");

    # restricciones de tamaño de rollos
    m.addConstrs((sum([w[i]*x[i,j] for i in I]) <= L*y[j] for j in J), "tam_rollo");

    # escribir el modelo a un archivo
    # m.write('cutting-stock.lp')

    # heurística para construir solución inicial
    # diccionario con valores de x y y (inicializar en cero)
    vx = {(i,j) : 0 for i in I for j in J} 
    vy = {j : 0 for j in J}

    ind = 0
    # satisfacer pedidos secuencialmente
    for i in I:
        # print(ind)
        # cantidad de items producidos del pedido i
        producidos = 0
        # cantidad de items que pueden cortarse de un rollo
        cant = int(L // w[i]) 
        # asignar rollos al pedido i hasta saitsfacer la demanda
        while producidos < b[i]:
            j = J[ind]
            vy[j]= 1
            vx[i,j] = cant
            producidos += cant
            ind += 1

    # cargar valores iniciales de x
    for i in I:
        for j in J:
            x[i,j].Start = vx[i,j]
            
    # cargar valores iniciales de y
    for j in J:
        y[j].Start = vy[j]

    # terminar al alcanzar un gap del 10%
    m.Params.MIPGap = 0.01
    
    # terminar luego de 180 segundos
    m.Params.TimeLimit = 180
    
    # resolver el modelo
    m.optimize()

    # # recuperar variables y_j cuyo valor sea 1:
    # for j in J:
    #    if y[j].getAttr('X') > 0.1:
    #        # recuperar valores x_ij asociados al rollo j y ponerlos en una lista
    #        L =[(int(x[i,j].getAttr('X')),i) for i in I if x[i,j].getAttr('X')>=0.9]
    #        # transformar la lista en una cadena de caracteres e imprimirla
    #        S = ['{:2}x{} '.format(k,i) for k,i in L]
    #        print ('[{}]'.format(''.join(S)))
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: código: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')